In [ ]:
from define import chandefs as chdef
from define import chanreps as chrep

In [ ]:
# chi_from_file = np.load("/Users/pavi/Documents/chbank/20_05_2020_11_07_46/physical/raw_up_6.5.npy")[1,:].reshape([4,4])
# ptm_from_file = np.load("/Users/pavi/Documents/chbank/20_05_2020_11_07_46/physical/up_6.5.npy")[1,:].reshape([4,4])
# np.save("/Users/pavi/Documents/chbank/20_05_2020_11_41_03/physical/pauli_0.929172_0.0218562_0.0307083_0.0182633.npy.npy", ptm_from_file.ravel()[np.newaxis,:])

In [ ]:
# Random channel in its Kraus representation
kraus = chdef.GetKraussForChannel("rand", 0.7)
print(kraus)

In [ ]:
# Converting the Kraus to Pauli transfer matrix
ptm_from_kraus = chrep.ConvertRepresentations(kraus, "krauss", "process")
print(ptm_from_kraus)

In [ ]:
# Converting to Choi
choi = chrep.ConvertRepresentations(kraus, "krauss", "choi")
# and then from Choi to Pauli transfer matrix
ptm_from_choi = chrep.ConvertRepresentations(choi, "choi", "process")
print(ptm_from_choi)

In [ ]:
# Norm of the difference of the matrices obtained by two different methods
print(np.linalg.norm(ptm_from_choi - ptm_from_kraus))

Matching the outputs of dnorm in chflow and qutip.

In [ ]:
import numpy as np
import qutip as qt
import cvxopt as cvx
import picos as pic

In [ ]:
def DiamondNorm(choi):
    # computes the diamond norm of the difference between an input Channel and another reference channel, which is by default, the identity channel
    # The semidefinite program outlined in Sec. 4 of DOI: 10.4086/toc.2009.v005a011 is used here.
    # See also: https://github.com/BBN-Q/matlab-diamond-norm/blob/master/src/dnorm.m
    # For some known types of channels, the Diamond norm can be computed efficiently
    # print("Function: dnorm")
    bell = np.zeros((4,4),dtype=np.complex128)
    bell[0,0]=0.5
    bell[0,3]=0.5
    bell[3,0]=0.5
    bell[3,3]=0.5
    diff = np.real((choi - bell + choi.conj().T - bell.conj().T))/2
    #### picos optimization problem
    prob = pic.Problem()
    # variables and parameters in the problem
    J = pic.Constant("J", cvx.matrix(diff))
    Z = pic.Constant("Z", value=0, shape=(2,2))
    rho = pic.HermitianVariable("rho", (2, 2))
    W = pic.HermitianVariable("W", (4, 4))
    # objective function (maximize the hilbert schmidt inner product -- denoted by '|'. Here A|B means trace(A^\dagger * B))
    prob.set_objective("max", J | W)
    # adding the constraints
    prob.add_constraint(W >> 0)
    prob.add_constraint(rho >> 0)
    prob.add_constraint(("I" | rho) == 1)
    prob.add_constraint((W - ((rho & Z) // (Z & rho))) << 0)
    # solving the problem
    prob.solve(verbosity=0, max_iterations=500)
    dnorm = np.double(prob) * 2
    # print("SDP dnorm = %.4e" % (dnorm))
    return dnorm

In [ ]:
channel = qt.rand_super(2)
choi_obj = qt.to_choi(channel)
choi_mat = choi_obj.full()/2
print("Random channel:\n{}".format(np.round(choi_mat,2)))

In [ ]:
isherm = np.round(choi_mat - choi_mat.T.conj(),2)
isuntr = np.round(np.trace(choi_mat),2)
eigvals = np.round(np.linalg.eigvals(choi_mat),2)
print("R - R^\dag = {}\nTrace(R) = {}\nEigenvalues = {}".format(isherm, isuntr, eigvals))

In [ ]:
qutip_dnorm = qt.dnorm(choi_obj)

In [ ]:
chflow_dnorm = DiamondNorm(choi_mat)

In [ ]:
print("qutip: {}, chflow = {}. Difference = {}.".format(qutip_dnorm, chflow_dnorm, abs(qutip_dnorm - chflow_dnorm)))

In [92]:
import numpy as np
from define import qcode as qc
from define import chanreps as chrep
from define import globalvars as gv
from define.QECCLfid import uncorrectable as uc

In [93]:
qcode = qc.QuantumErrorCorrectingCode("5qc")
qc.Load(qcode)
G = qc.GenerateGroup(qcode.S)

In [145]:
# chi_from_file = np.load("/Users/pavi/Documents/chbank/20_05_2020_14_05_10/physical/raw_up_6.5.npy")[1,:].reshape([4,4])
ptm_from_file = np.load("/Users/pavi/Documents/chbank/21_05_2020_17_42_43/physical/up_6.8.npy")[3,:].reshape([4,4])
# np.save("/Users/pavi/Documents/chbank/20_05_2020_11_41_03/physical/pauli_0.929172_0.0218562_0.0307083_0.0182633.npy.npy", ptm_from_file.ravel()[np.newaxis,:])

In [135]:
print("S\n{}\nL\n{}\nT\n{}".format(qcode.S,qcode.L,qcode.T))
print("Lookup\n{}".format(qcode.lookup[:,0]))

S
[[3 1 0 1 3]
 [3 3 1 0 1]
 [1 3 3 1 0]
 [0 1 3 3 1]]
L
[[1 1 1 1 1]
 [3 3 3 3 3]]
T
[[1 1 1 1 0]
 [1 0 0 0 1]
 [0 0 1 1 0]
 [1 1 1 0 1]]
Lookup
[0 1 3 1 3 3 1 2 1 3 3 2 1 2 2 2]


In [136]:
ptm_from_file

array([[1.        , 0.        , 0.        , 0.        ],
       [0.        , 0.8684932 , 0.        , 0.        ],
       [0.        , 0.        , 0.88517233, 0.        ],
       [0.        , 0.        , 0.        , 0.98137146]])

In [137]:
C=chrep.ConvertRepresentations(ptm_from_file, "process", "chi")

In [138]:
single_probs = np.tile(np.real(np.diag(C)),[qcode.N,1])
print("Single qubit probabilities\n{}\nSum\n{}".format(single_probs,np.sum(single_probs, axis=1)))
pu = uc.ComputeUnCorrProbUsingMinWt(single_probs, qcode)
print("p_u = {}".format(pu))

Single qubit probabilities
[[9.33759249e-01 4.87352675e-04 8.82691568e-03 5.69264830e-02]
 [9.33759249e-01 4.87352675e-04 8.82691568e-03 5.69264830e-02]
 [9.33759249e-01 4.87352675e-04 8.82691568e-03 5.69264830e-02]
 [9.33759249e-01 4.87352675e-04 8.82691568e-03 5.69264830e-02]
 [9.33759249e-01 4.87352675e-04 8.82691568e-03 5.69264830e-02]]
Sum
[1. 1. 1. 1. 1.]
p_u = 0.038028695253963685


In [113]:
pauliprobs = np.array([np.prod([single_probs[q,qcode.PauliOperatorsLST[p,q]] for q in range(qcode.N)]) for p in range(qcode.PauliOperatorsLST.shape[0])], dtype = np.double)
np.sum(pauliprobs)

1.0

In [147]:
infid = np.load("/Users/pavi/Documents/chbank/21_05_2020_17_42_43/metrics/infid_6.8_s3.npy")
print("Infidelity = {}".format(infid))

Infidelity = [0.06624075 0.02639643]


In [114]:
1-sum(pauliprobs[:2**(2*qcode.N-2*qcode.K)])

0.28995949888075234

In [63]:
blog=1 # 1 -> Z, 2 -> X, 3 -> Y
ptm = np.zeros(4*2**(qcode.N-qcode.K),dtype=np.double)
chrep.GetTransferMatrixElements(blog, pauliprobs, qcode, ptm)
p_code = [ptm[i] for i in range(blog*2**(qcode.N-qcode.K), (blog+1)*2**(qcode.N-qcode.K))]

In [64]:
p_code

[0.7949879508698532,
 0.8911415594019021,
 0.8911415594019019,
 0.757550478665178,
 0.8911415594019021,
 0.8119643998778334,
 0.757550478665178,
 0.757550478665178,
 0.8911415594019021,
 0.8119643998778333,
 0.8119643998778333,
 0.8119643998778333,
 0.757550478665178,
 0.8119643998778334,
 0.757550478665178,
 0.891141559401902]

In [65]:
# Construct full process matrix for a channel, given its normalizer
ilog = 3 # 1 -> X, 2 -> Y, 3 -> Z
nstabs = 2**(qcode.N-qcode.K)
processes = np.tile(ptm_from_file, [qcode.N,1,1])
full_process = np.zeros((nstabs, nstabs), dtype=np.double)
for s in range(nstabs):
    full_process[s, s] = np.real(qcode.normphases[ilog,s] * qcode.normphases[ilog,s]) * np.prod([processes[q, qcode.normalizer[ilog,s,q], qcode.normalizer[ilog,s,q]] for q in range(qcode.N)])
c_code = [dg for dg in np.diag(full_process)]

In [66]:
c_code

[0.7949879508698535,
 0.8911415594019022,
 0.8911415594019022,
 0.7575504786651783,
 0.8911415594019022,
 0.8119643998778338,
 0.7575504786651783,
 0.7575504786651783,
 0.8911415594019022,
 0.8119643998778338,
 0.8119643998778338,
 0.8119643998778338,
 0.7575504786651783,
 0.8119643998778338,
 0.7575504786651783,
 0.8911415594019022]

In [67]:
# p_code - c_code
np.diag(full_process)-ptm[range(blog*2**(qcode.N-qcode.K), (blog+1)*2**(qcode.N-qcode.K))]

array([3.33066907e-16, 1.11022302e-16, 3.33066907e-16, 3.33066907e-16,
       1.11022302e-16, 3.33066907e-16, 3.33066907e-16, 3.33066907e-16,
       1.11022302e-16, 4.44089210e-16, 4.44089210e-16, 4.44089210e-16,
       3.33066907e-16, 3.33066907e-16, 3.33066907e-16, 2.22044605e-16])

In [160]:
pu = np.load("/Users/pavi/Documents/chbank/23_05_2020_00_01_02/results/phy_uncorr.npy")
infids = np.load("/Users/pavi/Documents/chbank/23_05_2020_00_01_02/results/log_infid.npy")

In [161]:
# pu = uc.ComputeUnCorrProbUsingMinWt(single_probs, qcode)

In [162]:
pu

array([3.70945347e-02, 3.62979904e-02, 4.19729915e-02, 5.59058025e-02,
       4.16565699e-02, 4.30236788e-02, 5.39683298e-02, 3.42774828e-02,
       3.97070357e-02, 3.97340814e-02, 3.09070011e-02, 4.47272295e-02,
       3.29763994e-02, 3.63894962e-02, 2.98621101e-02, 4.47674502e-02,
       3.15452757e-02, 2.70838625e-02, 4.90476041e-02, 2.79597264e-02,
       3.18054132e-02, 4.03246270e-02, 2.82109500e-02, 4.38384740e-02,
       3.81486839e-02, 2.28034929e-02, 3.73543643e-02, 3.51290308e-02,
       3.10180173e-02, 2.76685198e-02, 3.50376500e-02, 3.08569257e-02,
       3.48502577e-02, 2.44012577e-02, 2.99192223e-02, 2.06569368e-02,
       2.46418811e-02, 3.04695812e-02, 4.39490642e-02, 3.47983021e-02,
       1.99444516e-02, 2.92275230e-02, 2.67545416e-02, 2.70552002e-02,
       2.64783144e-02, 2.69698858e-02, 3.00017519e-02, 3.27240764e-02,
       2.66976975e-02, 2.96962130e-02, 1.97757964e-02, 3.18559331e-02,
       2.97400705e-02, 2.90633209e-02, 2.84333765e-02, 2.28063784e-02,
      

In [164]:
np.where(np.abs(pu-infids[:,1]) > 10E-15, pu-infids[:,1], 0)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [165]:
np.argmax(np.abs(pu-infids[:,1]))

401

In [156]:
infids[33,1]

0.024401257739171136607

In [142]:
qcode.lookup

array([[0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 1, 0],
       [3, 1, 3, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 0, 0],
       [3, 1, 0, 0, 3, 0, 0],
       [3, 1, 0, 0, 0, 0, 3],
       [1, 1, 0, 1, 0, 0, 0],
       [2, 1, 0, 0, 2, 0, 0],
       [1, 1, 0, 0, 0, 0, 1],
       [3, 1, 0, 3, 0, 0, 0],
       [3, 1, 0, 0, 0, 3, 0],
       [2, 1, 0, 0, 0, 2, 0],
       [1, 1, 1, 0, 0, 0, 0],
       [2, 1, 0, 0, 0, 0, 2],
       [2, 1, 2, 0, 0, 0, 0],
       [2, 1, 0, 2, 0, 0, 0]], dtype=int8)

In [143]:
qcode.S

array([[3, 1, 0, 1, 3],
       [3, 3, 1, 0, 1],
       [1, 3, 3, 1, 0],
       [0, 1, 3, 3, 1]], dtype=int8)

In [144]:
qcode.T

array([[1, 1, 1, 1, 0],
       [1, 0, 0, 0, 1],
       [0, 0, 1, 1, 0],
       [1, 1, 1, 0, 1]], dtype=int8)

In [175]:
uncorr_backup = np.load("/Users/pavi/Documents/chbank/06_05_2020_20_07_32/results/phy_uncorr_backup.npy")
uncorr = np.load("/Users/pavi/Documents/chbank/06_05_2020_20_07_32/results/phy_uncorr.npy")
infids = np.load("/Users/pavi/Documents/chbank/06_05_2020_20_07_32/results/log_infid.npy")

In [177]:
infids.shape

(48, 3)

In [178]:
uncorr-infids[:,1]

array([3.66112334e-02, 2.84429037e-02, 3.34283853e-02, 1.18989968e-02,
       1.46786135e-03, 5.57060669e-03, 2.56434253e-02, 9.43696298e-03,
       1.72293316e-03, 9.94555629e-03, 1.02786158e-02, 2.44227210e-03,
       1.67454785e-02, 5.80144622e-03, 1.41206311e-03, 2.52274783e-03,
       4.78128638e-03, 3.25009452e-03, 9.21349962e-05, 2.71426370e-03,
       8.07538669e-04, 7.54950472e-04, 1.46332957e-03, 1.04836278e-03,
       7.08279004e-05, 1.92530884e-04, 5.51690412e-04, 2.07404137e-04,
       1.29927381e-04, 2.08836323e-04, 7.41100343e-05, 1.21375927e-04,
       3.53453288e-05, 1.14010797e-05, 4.40033807e-05, 3.39735306e-05,
       1.72586736e-05, 2.85542588e-06, 1.08027740e-05, 6.05664287e-06,
       8.95851037e-07, 1.71025249e-05, 1.24214772e-06, 1.07923890e-06,
       6.44877170e-06, 1.15415003e-06, 1.28575015e-05, 5.04200281e-06],
      dtype=float128)

In [174]:
uncorr_backup-uncorr

array([ 3.28636575e-02,  1.26213267e-03, -2.24689080e-03, -6.41839980e-03,
       -3.04666843e-04,  3.03911453e-03,  5.13515386e-03,  2.43438981e-03,
        1.55865313e-03, -5.80934321e-03, -6.09121327e-04, -2.70308307e-04,
        2.67223017e-03, -3.44706010e-03, -6.70914535e-05, -1.54738517e-03,
       -2.58542112e-03, -1.32265110e-03,  1.69498016e-05, -3.44612358e-04,
       -3.42386237e-04, -1.41127253e-04, -1.86362414e-04, -6.79147569e-04,
        5.22358164e-05, -5.07445985e-05, -2.14861682e-04,  2.10762485e-04,
       -5.69609352e-05, -1.76639836e-04, -8.78325885e-07, -3.27555803e-06,
        3.10503476e-06, -5.10863801e-06,  7.60129993e-05, -1.99477659e-05,
        7.40563197e-07,  7.60805141e-08,  3.30004793e-06,  1.50723055e-06,
       -1.43483880e-07,  4.16963536e-06, -3.22516121e-07, -3.98288292e-07,
        8.71269666e-06,  1.36646646e-06, -5.06204054e-06, -2.42722490e-06])